In [ ]:
# Cargar las librerías
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Usamos una variable para indicar la URL
BASE = "https://api-colombia.com/api/v1"

In [ ]:
# Llamamos los datos de la API
def get_json(path):
    r = requests.get(f"{BASE}/{path}", timeout=30)
    r.raise_for_status()
    return r.json()

In [ ]:
# Extraemos Datos (dos veces) Datos base ---
departments = pd.DataFrame(get_json("Department"))
airports = pd.DataFrame(get_json("Airport"))

In [ ]:
departments

In [ ]:
airports

In [ ]:
# --- 2) Limpiar nombres de columnas para unión ---
# Ojo: en airports el departamento es "deparmentId" y en departments es "id"
if "deparmentId" in airports.columns:
    airports.rename(columns={"deparmentId": "departmentId"}, inplace=True)

In [ ]:
# --- 3) Contar aeropuertos por departamento ---
airports_count = airports.groupby("departmentId").size().reset_index(name="num_airports")

In [ ]:
airports_count

In [ ]:
# --- 4) Unir con datos de departamentos ---
df = departments.merge(airports_count, left_on="id", right_on="departmentId", how="left")
df["num_airports"] = df["num_airports"].fillna(0)

In [ ]:
df

In [ ]:
# --- 5) Ranking simple: población y aeropuertos ---
ranking = df[["name", "population", "num_airports"]].sort_values(
    ["population", "num_airports"], ascending=[False, False]
)
print(ranking.head(10))

In [ ]:
# --- 6) Visualización rápida ---
ranking.head(10).set_index("name")[["population", "num_airports"]].plot(
    kind="bar", secondary_y="population", figsize=(10,6),
    title="Top 10 departamentos: mercado (población) y logística (aeropuertos)"
)
plt.ylabel("Número de aeropuertos")
plt.show()